In [23]:
import pandas as pd
import cv2
import numpy as np
import glob
import tensorflow as tf
from tensorflow import keras
import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers
from keras.layers import Input, Dense
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from datetime import datetime
from keras.models import model_from_json
# from keras import backend as K
%load_ext tensorboard
logdir = "tensorlogs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [13]:
training_data = []
label = 0
flag = 0
my_dict = {}
for i in glob.glob("/home/shivasankaran/datasets/equation solver/data/extracted_images/*"):
    symbol = i.split("/")[-1]
#     if(symbol not in my_dict.keys()):
    my_dict[label] = symbol
    label+=1
    
    print("Currently in : {} and a label {}:".format(i,label))
    data = []
    count = 0
    for image_path in glob.glob(i+"/*"):
        count+=1
        if(count>4000):
            print("4000 samples reaced moving to the next folder")
            break
            
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        gray = ~gray
        if gray is not None:
            thresh,binary = cv2.threshold(gray,115,255,cv2.THRESH_BINARY_INV)
            ctrs,_ = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            cnt=sorted(ctrs, key=lambda ctr: cv2.contourArea(ctr),reverse = True)
            x_max,y_max,w_max,h_max = cv2.boundingRect(cnt[0])
            cropped = binary[y_max:y_max+h_max+10, x_max:x_max+w_max+10]
            resized = cv2.resize(cropped,(28,28))
            reshaped = np.reshape(resized,(784,1))
            data.append(reshaped)
    
    
    if(flag == 0):
        training_data = data
        for j in range(len(data)):
            training_data[j] = np.append(training_data[j],str(label))
        flag = 1
    else:
        for j in range(len(data)):
            data[j] = np.append(data[j],str(label))
        training_data = np.concatenate((training_data,data))
    print("Training data has {} samples and each sample is of length{}".format(len(training_data),(training_data[-1].shape)))
    
    

Currently in : /home/shivasankaran/datasets/equation solver/data/extracted_images/Delta and a label 1:
Training data has 137 samples and each sample is of length(785,)
Currently in : /home/shivasankaran/datasets/equation solver/data/extracted_images/pi and a label 2:
Training data has 2469 samples and each sample is of length(785,)
Currently in : /home/shivasankaran/datasets/equation solver/data/extracted_images/int and a label 3:
Training data has 5211 samples and each sample is of length(785,)
Currently in : /home/shivasankaran/datasets/equation solver/data/extracted_images/5 and a label 4:
Training data has 8756 samples and each sample is of length(785,)
Currently in : /home/shivasankaran/datasets/equation solver/data/extracted_images/gt and a label 5:
Training data has 9014 samples and each sample is of length(785,)
Currently in : /home/shivasankaran/datasets/equation solver/data/extracted_images/sum and a label 6:
Training data has 11703 samples and each sample is of length(785,)


Training data has 98596 samples and each sample is of length(785,)
Currently in : /home/shivasankaran/datasets/equation solver/data/extracted_images/T and a label 48:
Training data has 101870 samples and each sample is of length(785,)
Currently in : /home/shivasankaran/datasets/equation solver/data/extracted_images/3 and a label 49:
4000 samples reaced moving to the next folder
Training data has 105870 samples and each sample is of length(785,)
Currently in : /home/shivasankaran/datasets/equation solver/data/extracted_images/! and a label 50:
Training data has 107170 samples and each sample is of length(785,)
Currently in : /home/shivasankaran/datasets/equation solver/data/extracted_images/forall and a label 51:
Training data has 107215 samples and each sample is of length(785,)
Currently in : /home/shivasankaran/datasets/equation solver/data/extracted_images/infty and a label 52:
Training data has 108998 samples and each sample is of length(785,)
Currently in : /home/shivasankaran/dat

In [5]:
print(len(training_data))
print(len(training_data[0]))

178553
785


In [4]:
df=pd.read_csv('/home/shivasankaran/datasets/equation solver/train_final.csv',index_col=False)


In [6]:
labels=df['784']
labels = np.array(labels)
for i in range(len(labels)):
    labels[i] = str(int(labels[i])-1)

cat=to_categorical(labels,num_classes=82)


df.drop(df.columns[[784]],axis=1,inplace=True)
df.head()
l=[]
for i in range(178553):
    l.append(np.array(df[i:i+1]).reshape(1,28,28))

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [28]:
tf.keras.backend.set_image_data_format("channels_first")

model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(1 , 28, 28), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(82, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [30]:
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)

model.fit(np.array(l), cat, epochs=50, batch_size=200,shuffle=True,verbose=1,callbacks=[tensorboard_callback])

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5

Epoch 1/50
893/893 [==============================] - 5s 6ms/step - loss: 0.6604 - accuracy: 0.7954
Epoch 2/50
893/893 [==============================] - 5s 5ms/step - loss: 0.6098 - accuracy: 0.8111
Epoch 3/50
893/893 [==============================] - 5s 5ms/step - loss: 0.5601 - accuracy: 0.8239
Epoch 4/50
893/893 [==============================] - 5s 5ms/step - loss: 0.5194 - accuracy: 0.8344
Epoch 5/50
893/893 [==============================] - 5s 5ms/step - loss: 0.4918 - accuracy: 0.8437
Epoch 6/50
893/893 [==============================] - 5s 5ms/step - loss: 0.4728 - accuracy: 0.8487
Epoch 7/50
893/893 [==============================] - 5s 5ms/step - loss: 0.4767 - accuracy: 0.8472


In [21]:
test_images = []
for i in glob.glob("/home/shivasankaran/datasets/equation solver/data/extracted_images/*"):
    test_images.append((glob.glob(i+"/*")[-100:],i.split("/")[-1]))

In [26]:
# test_image = "/home/shivasankaran/datasets/equation solver/data/extracted_images/3/3_108.jpg"
tot_no = 0
correct=0
for i in range(len(test_images)):

    images = test_images[i][0]
    symbol = test_images[i][1]
    for image in images:
        tot_no+=1
        img = cv2.imread(image)
        gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        resized_img = cv2.resize(gray_img,(28,28))

#         plt.imshow(resized_img, cmap = 'binary', interpolation = 'bicubic')
#         plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
#         plt.show()

        resized_img = np.reshape(resized_img,(1,1,28,28))

        result = my_model.predict_classes(resized_img)

        if(my_dict[result[0]] == symbol):
            correct+=1
        else:
            print("predicted wrong!!  correct:{} mode predicted:{}".format(symbol,my_dict[result[0]]))

print("FINAL ACCURACY:",(correct/tot_no)*100)

predicted wrong!!  correct:pi mode predicted:T
predicted wrong!!  correct:5 mode predicted:S
predicted wrong!!  correct:5 mode predicted:S
predicted wrong!!  correct:5 mode predicted:S
predicted wrong!!  correct:5 mode predicted:S
predicted wrong!!  correct:5 mode predicted:S
predicted wrong!!  correct:5 mode predicted:S
predicted wrong!!  correct:5 mode predicted:3
predicted wrong!!  correct:5 mode predicted:S
predicted wrong!!  correct:5 mode predicted:S
predicted wrong!!  correct:times mode predicted:X
predicted wrong!!  correct:times mode predicted:X
predicted wrong!!  correct:times mode predicted:X
predicted wrong!!  correct:times mode predicted:X
predicted wrong!!  correct:times mode predicted:X
predicted wrong!!  correct:times mode predicted:X
predicted wrong!!  correct:times mode predicted:X
predicted wrong!!  correct:times mode predicted:X
predicted wrong!!  correct:times mode predicted:X
predicted wrong!!  correct:times mode predicted:X
predicted wrong!!  correct:times mode p

predicted wrong!!  correct:b mode predicted:6
predicted wrong!!  correct:b mode predicted:H
predicted wrong!!  correct:M mode predicted:X
predicted wrong!!  correct:j mode predicted:y
predicted wrong!!  correct:j mode predicted:A
predicted wrong!!  correct:j mode predicted:f
predicted wrong!!  correct:j mode predicted:phi
predicted wrong!!  correct:in mode predicted:e
predicted wrong!!  correct:in mode predicted:e
predicted wrong!!  correct:in mode predicted:e
predicted wrong!!  correct:in mode predicted:e
predicted wrong!!  correct:2 mode predicted:l
predicted wrong!!  correct:2 mode predicted:-
predicted wrong!!  correct:2 mode predicted:z
predicted wrong!!  correct:2 mode predicted:z
predicted wrong!!  correct:2 mode predicted:N
predicted wrong!!  correct:2 mode predicted:z
predicted wrong!!  correct:2 mode predicted:z
predicted wrong!!  correct:2 mode predicted:z
predicted wrong!!  correct:2 mode predicted:d
predicted wrong!!  correct:2 mode predicted:geq
predicted wrong!!  correct

predicted wrong!!  correct:, mode predicted:(
predicted wrong!!  correct:, mode predicted:ascii_124
predicted wrong!!  correct:, mode predicted:forward_slash
predicted wrong!!  correct:, mode predicted:prime
predicted wrong!!  correct:, mode predicted:)
predicted wrong!!  correct:, mode predicted:)
predicted wrong!!  correct:, mode predicted:ascii_124
predicted wrong!!  correct:, mode predicted:)
predicted wrong!!  correct:, mode predicted:forward_slash
predicted wrong!!  correct:, mode predicted:)
predicted wrong!!  correct:, mode predicted:)
predicted wrong!!  correct:, mode predicted:ascii_124
predicted wrong!!  correct:, mode predicted:(
predicted wrong!!  correct:, mode predicted:ascii_124
predicted wrong!!  correct:, mode predicted:)
predicted wrong!!  correct:, mode predicted:ascii_124
predicted wrong!!  correct:, mode predicted:1
predicted wrong!!  correct:, mode predicted:1
predicted wrong!!  correct:8 mode predicted:l
predicted wrong!!  correct:v mode predicted:p
predicted wr

predicted wrong!!  correct:forward_slash mode predicted:div
predicted wrong!!  correct:forward_slash mode predicted:,
predicted wrong!!  correct:forward_slash mode predicted:(
predicted wrong!!  correct:forward_slash mode predicted:,
predicted wrong!!  correct:forward_slash mode predicted:div
predicted wrong!!  correct:forward_slash mode predicted:(
predicted wrong!!  correct:forward_slash mode predicted:div
predicted wrong!!  correct:forward_slash mode predicted:,
predicted wrong!!  correct:forward_slash mode predicted:1
predicted wrong!!  correct:forward_slash mode predicted:1
predicted wrong!!  correct:forward_slash mode predicted:,
predicted wrong!!  correct:forward_slash mode predicted:)
predicted wrong!!  correct:forward_slash mode predicted:div
predicted wrong!!  correct:forward_slash mode predicted:div
predicted wrong!!  correct:forward_slash mode predicted:,
predicted wrong!!  correct:forward_slash mode predicted:,
predicted wrong!!  correct:forward_slash mode predicted:ascii_

In [47]:
# from keras.models import model_to_json
model_json = model.to_json()
with open("/home/shivasankaran/saved_models/eqsolver/eqnsolver_v1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/home/shivasankaran/saved_models/eqsolver/eqnsolver_v1.h5")